###### Data Jobs Analysis 
[Go back to the main page](./Data_Jobs_Analysis.ipynb)

---------

## Data Offers Over Time

-------


In [182]:
import datetime
import psycopg2
import pandas as pd
import ipywidgets as widgets
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

plotly.offline.init_notebook_mode(connected=True) # run at the start of every ipython notebook to use plotly.offline

%matplotlib notebook
%matplotlib inline

In [183]:
conn = psycopg2.connect("dbname=jobsbg")
ads_weekly = pd.read_sql_query('SELECT * FROM v_data_offers_count_weekly', conn, index_col='week_ts')
ads_monthly = pd.read_sql_query('SELECT * FROM v_data_offers_count_monthly', conn, index_col='month_ts')

ads_weekly = ads_weekly[1:-1]
ads_monthly = ads_monthly[1:-1]

ads_weekly.index = pd.to_datetime(ads_weekly.index)
ads_monthly.index = pd.to_datetime(ads_monthly.index)

In [210]:
ads_weekly.head(1), ads_monthly.head(1)

(            subm_count
 week_ts               
 2017-10-02          24,             subm_count
 month_ts              
 2017-10-01          91)

### Implementing the Chart in Plotly

Preparing two traces to represent our levels of aggregation for this chart. Which one will be visible will be controlled via buttons.

In [136]:
weekly_bar = go.Bar(
    x = [week for week in ads_weekly.index],
    y = [value for value in ads_weekly.subm_count],
    hoverinfo = 'x+y',
    showlegend = False,
    marker = dict(
        line = dict(width=1, color='#7F331B'),
        color = '#BF4D28',
        opacity = 0.8,
    ),
    visible = True,
)

monthly_bar = go.Bar(
    x = [month for month in ads_monthly.index],
    y = [value for value in ads_monthly.subm_count],
    hoverinfo = 'x+y',
    showlegend = False,
    marker = dict(
        line = dict(width=1, color='#7F331B'),
        color = '#BF4D28',
        opacity = 0.8,
    ),
    visible = False,
)

data = [weekly_bar, monthly_bar]

 Plotly buttons and defined interactions to make the chart interactive

In [194]:
updatemenus = list([
    dict(
        type='buttons',
        direction='right',
        xanchor = 'center',
        x = 0.5,
        yanchor = 'top',
        y = 1.09,
        active=0,
        showactive = True,
        bgcolor = '#FFFAF0',
        bordercolor = '#594B42',
        font = dict(size=15),
        buttons=list([
            dict(
                label = 'Weekly',
                method = 'update',
                args = [
                    {'visible': [True, False]},
                    {'title': 'Total Data Related Job Offers (Weekly)',
                    'xaxis.hoverformat': 'Week %W, %Y'}
                ]),
            dict(
                label = 'Monthly',
                method = 'update',
                args = [
                    {'visible': [False, True]},
                    {'title': 'Total Data Related Job Offers (Monthly)',
                    'xaxis.hoverformat': '%B \'%y'}
                ]),
        ]),
    )
])

In [216]:
layout = go.Layout(
    paper_bgcolor = '#FFFAF0',            
    plot_bgcolor = '#FFFAF0',
    title = 'Total Data Related Job Offers (Weekly)',
    titlefont = dict(
        color = '#594B42',
        family = 'FiraSansCondensedRegular',
        size = 22,
    ),
    font = dict(
        color = '#40362F',
        family = 'FiraSansCondensedRegular',
        size = 14,
    ),
    xaxis = dict(
        type = 'date',
        fixedrange = True,
        hoverformat = 'Week %W, %Y',
        ticks = 'outside',
        tickmode = 'auto',
        zerolinecolor = '#594B42',
    ),
    yaxis = dict(
        title = 'Num. submissions',
        fixedrange = True,
        hoverformat = '{:,}',
        tickformat = '{:,}',
        ticks = 'outside',
        tickwidth = 1,
        gridcolor = '#D9D4CC',
        zerolinecolor = '#594B42',
    ),
    updatemenus=updatemenus,
)

In [239]:
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename = 'data_offers_over_time.html')

In [238]:
# Uncomment the line below to export an HTML version of the chart.
# plotly.offline.plot(fig, filename = 'data_offers_over_time.html')

'file:///data/WORKSPACE/jpynb_Employment_Trends_Bulgaria/workbooks/data_offers_over_time.html'

In [1]:
from IPython.core.display import HTML
with open('../resources/styles/datum.css', 'r') as f:
    style = f.read()
HTML(style)